In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import datetime   
df = pd.read_excel('https://github.com/manoelgadi/FA/raw/main/ethusd5min.xlsx')
df.set_index('datetime',drop=True,inplace=True)
df.index = pd.to_datetime(df.index)
df = df.drop(df.columns[0], axis=1)
df = df.resample('1H').last()
df['slow_ma'] = df['close'].rolling(40).mean()
df['fast_ma'] = df['close'].rolling(120).mean()
df['position'] = np.where(df['fast_ma']>df['slow_ma'],1,-1)
df['entry'] = np.where(df['position'].shift(1)!=df['position'],np.where(df['position'].shift(1)==-1,1,-1),0)
df_entries = df.loc[df['entry']!=0]
df_entries = df_entries.dropna()
df_entries['return'] = np.log(df_entries['close']/df_entries['close'].shift(1))
df_entries['strategy_return'] = df_entries['return'] * df_entries['position'].shift(1)
df_entries['cum_strat_ret'] = (1+df_entries['strategy_return']).cumprod()
initial_capital = 100000
df_entries['capital'] = initial_capital*df_entries['cum_strat_ret']
winning_rate = len(df_entries[df_entries['strategy_return']>0])/len(df_entries)
losing_rate = 1 - winning_rate
profit_factor = df_entries
avg_gain = df_entries[df_entries['strategy_return']>0]['strategy_return'].mean()
max_gain = df_entries['strategy_return'].max()
avg_loss = df_entries[df_entries['strategy_return']<=0]['strategy_return'].mean()
max_loss = df_entries['strategy_return'].min()
expectancy = abs((winning_rate*avg_gain)/(losing_rate*avg_loss))
print("winning rate : {0:.2f} %".format(winning_rate*100))
print("Losing rate : {0:.2f} %".format(losing_rate*100))
print("maximum gain : {0:.2f} %".format(max_gain*100))
print("maximum loss : {0:.2f} %".format(max_loss*100))
print("average gain: {0:.2f} %".format(avg_gain*100))
print("average loss: {0:.2f} %".format(avg_loss*100))
print("expectancy: {0:.2f}".format(expectancy))